In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ramil\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ramil\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
from nltk.stem import PorterStemmer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [4]:
df = pd.read_csv('labeledTrainData.tsv', sep='\t', encoding='utf-8')

In [5]:
texts = df.review.tolist()
sentiment = df.sentiment.tolist()

In [6]:
amount = 1000
X_train, X_test, y_train, y_test = train_test_split(texts[:amount], sentiment[:amount])

In [8]:
# clean text
import re
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer

cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')

def cleanhtml(text):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', text)
    return cleantext

def word_tokenize(text):
    tokenizer = RegexpTokenizer(r'\w+')#fix
    clean = tokenizer.tokenize(text)
    return clean
   
def stop_words(tokenize_words):
    stopWords = set(stopwords.words('english'))
    wordsFiltered = []
    for w in tokenize_words:
        if w not in stopWords:
            wordsFiltered.append(w.lower())
    return wordsFiltered


def stem(text_arr):
    ps = PorterStemmer()
    stem_arr = []
    for word in text_arr:
        stem_arr.append(ps.stem(word))
    return stem_arr

def preprocess_text(text):
    #clean html 
    clean = cleanhtml(text)
    
    # tokenize and remove punctuation 
    tokenize = word_tokenize(clean)
    
    # stop words
    stop = stop_words(tokenize)
    
    # stem 
    stem_arr = stem(stop)
    
    return stem_arr


def preprocess_texts(texts):
    tmp_set = set()
    for text in texts:
        arr = preprocess_text(text) 
        tmp_set.update(arr)
    return tmp_set

print(preprocess_text(df.to_dict('records')[0]['review']))

['with', 'stuff', 'go', 'moment', 'mj', 'start', 'listen', 'music', 'watch', 'odd', 'documentari', 'watch', 'the', 'wiz', 'watch', 'moonwalk', 'mayb', 'want', 'get', 'certain', 'insight', 'guy', 'thought', 'realli', 'cool', 'eighti', 'mayb', 'make', 'mind', 'whether', 'guilti', 'innoc', 'moonwalk', 'part', 'biographi', 'part', 'featur', 'film', 'rememb', 'go', 'see', 'cinema', 'origin', 'releas', 'some', 'subtl', 'messag', 'mj', 'feel', 'toward', 'press', 'also', 'obviou', 'messag', 'drug', 'bad', 'kay', 'visual', 'impress', 'cours', 'michael', 'jackson', 'unless', 'remot', 'like', 'mj', 'anyway', 'go', 'hate', 'find', 'bore', 'some', 'may', 'call', 'mj', 'egotist', 'consent', 'make', 'movi', 'but', 'mj', 'fan', 'would', 'say', 'made', 'fan', 'true', 'realli', 'nice', 'the', 'actual', 'featur', 'film', 'bit', 'final', 'start', '20', 'minut', 'exclud', 'smooth', 'crimin', 'sequenc', 'joe', 'pesci', 'convinc', 'psychopath', 'power', 'drug', 'lord', 'whi', 'want', 'mj', 'dead', 'bad', 'be

In [9]:
# tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
# print(train_texts)
corpus = X_train

vocab_tf = preprocess_texts(corpus)

tfidf = TfidfVectorizer(stop_words = 'english', vocabulary=vocab_tf)
X = tfidf.fit_transform(corpus)
X_dense = X.todense()

# print(tfidf.get_feature_names())
X_test_transform = tfidf.transform(X_test)
X_test_transform_dense = X_test_transform.todense()

In [10]:
# LinearRegression
from sklearn.linear_model import LogisticRegression
import numpy as np

clf = LogisticRegression(random_state=0).fit(X_dense, y_train)
pred = clf.predict(X_test_transform_dense)
print(classification_report(pred, y_test[:amount]))

              precision    recall  f1-score   support

           0       0.81      0.76      0.79       136
           1       0.74      0.79      0.76       114

    accuracy                           0.78       250
   macro avg       0.78      0.78      0.78       250
weighted avg       0.78      0.78      0.78       250



In [11]:
# DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier

dclf = DecisionTreeClassifier(random_state=0)

dclf.fit(X_dense, y_train)      # Use fit method on the train data

dtc_pred = dclf.predict(X_test_transform_dense)   # Predict the target class of test data
print(classification_report(dtc_pred, y_test[:amount])) 

              precision    recall  f1-score   support

           0       0.64      0.67      0.65       123
           1       0.66      0.64      0.65       127

    accuracy                           0.65       250
   macro avg       0.65      0.65      0.65       250
weighted avg       0.65      0.65      0.65       250



In [12]:
# KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier


neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_dense, y_train)
neigh_pred = neigh.predict(X_test_transform_dense)
print(classification_report(neigh_pred, y_test[:amount]))

              precision    recall  f1-score   support

           0       0.67      0.61      0.64       140
           1       0.56      0.62      0.59       110

    accuracy                           0.62       250
   macro avg       0.61      0.62      0.61       250
weighted avg       0.62      0.62      0.62       250

